##Initialize D-Wave

In [ ]:
pip install dwave-ocean-sdk

In [ ]:
!dwave setup

In [ ]:
!dwave config create

In [ ]:
!dwave config inspect

In [ ]:
!dwave ping

In [5]:
# ------ Import necessary packages ----
from collections import defaultdict

from dwave.system.samplers import DWaveSampler, LeapHybridSampler
from dwave.system.composites import EmbeddingComposite, FixedEmbeddingComposite
import networkx as nx
import random
import time
import numpy as np 

import matplotlib
matplotlib.use("agg")
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

In [6]:
# Import the Dwave packages dimod and neal
import dimod
import neal
from dimod.reference.samplers import ExactSolver

In [11]:
def stopper(func):
  start = time.time()
  returnVal = func()
  stop = time.time()
  print("Elapsed time:", stop-start)
  return returnVal

##Graph generation methods

In [2]:
# ------- Graph creation -------
def createRandomGraphClusters(n, p, q, K=2):
  G=nx.empty_graph(K*n)

  #Edges across
  for k1 in range(K):
    for k2 in range(K):
      if (k1!=k2):
        for u in range(n*k1,n*(k1+1)):
          for v in range(n*k2,n*(k2+1)):
            if (random.random()<p):
              G.add_edge(u,v)

  #Edges within
  for k1 in range(K):
    for u in range(n*k1,n*(k1+1)):
      for v in range(u+1,n*(k1+1)):
        if (random.random()<q):
          G.add_edge(u,v)

  #Assign color to each node
  for k in range(K):
    for i in range(0,n):
      G.nodes[n*k+i]['c']=k

  return G


def addWeightsToGraph(G):
  across_edges = [(u, v) for u, v in G.edges if G.nodes[u]['c']!=G.nodes[v]['c']]
  within_edges = [(u, v) for u, v in G.edges if G.nodes[u]['c']==G.nodes[v]['c']]
  weightWithin = 0.0
  weightAcross = 0.0
  for (u, v) in across_edges:
    rand=random.random()
    weightAcross+=rand
    G.edges[u, v]['weight']=rand

  for (u, v) in within_edges:    
    rand=random.random()
    weightWithin+=rand
    G.edges[u, v]['weight']=rand

  print("weightAcross:", weightAcross)
  print("weightWithin:", weightWithin)
  return G

###Deprecated graph generation

In [3]:
def createRandomGraph(n, p, q):
  G=nx.empty_graph(2*n)

  #Edges across
  for i in range(0,n):
    for j in range(n,2*n):
      if (random.random()<p):
        G.add_edge(i,j)

  #Edges within
  for i in range(0,n):
    for j in range(0,n):
      if (random.random()<q):
        G.add_edge(i,j)
      if (random.random()<q):
        G.add_edge(n+i,n+j)
  
  for i in range(0,n):
    G.nodes[i]['c']=0
    G.nodes[n+i]['c']=1

  return G

def createRandomGraphWeighted(n, p, q):
  G=createRandomGraph(n, p, q)
  across_edges = [(u, v) for u, v in G.edges if G.nodes[u]['c']!=G.nodes[v]['c']]
  within_edges = [(u, v) for u, v in G.edges if G.nodes[u]['c']==G.nodes[v]['c']]
  weightWithin = 0.0
  weightAcross = 0.0
  for (u, v) in across_edges:
    rand=random.random()
    weightAcross+=rand
    G.edges[u, v]['weight']=rand

  for (u, v) in within_edges:    
    rand=random.random()
    weightWithin+=rand
    G.edges[u, v]['weight']=rand

  print("weightAcross:", weightAcross)
  print("weightWithin:", weightWithin)
  return G

##Construct graph

In [ ]:
# ------- Set up our graph -------

# Create empty graph
#G = nx.Graph()
N = 500
K = 2

#G = createRandomGraph(N,0.5,0.1)
#G = createRandomGraphClusters(N, 0.001, 0.99, K)
G = createRandomGraphClusters(N, 0.8, 0.1, K)
addWeightsToGraph(G)

#G = createRandomGraphWeighted(N,0.9,0.2)

# Add edges to the graph (also adds nodes)
#G.add_edges_from([(1,2),(1,3),(2,4),(3,4),(3,5),(4,5)])
#G.add_edges_from([(1,2,weight=0.1),(1,3,weight=0.1),(2,4,weight=0.1),(3,4,weight=0.1),(3,5,weight=0.1),(4,5,weight=0.1)])


#G= nx.circular_ladder_graph(11)
#G= nx.fast_gnp_random_graph(1000,0.2)

arr = np.arange(K)
color_map = np.repeat(arr, N)

In [ ]:
# ------- Set up our graph -------
# Graph for max-cut
N = 100
K = 4

G = createRandomGraphClusters(N, 0.5, 0.1, K)
addWeightsToGraph(G)

arr = np.arange(K)
color_map = np.repeat(arr, N)

In [ ]:
cut_edges = [(u, v) for u, v in G.edges if color_map[u]!=color_map[v]]
uncut_edges = [(u, v) for u, v in G.edges if color_map[u]==color_map[v]]

pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, nodelist=G.nodes, node_color=color_map)
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=1)
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=1)
#nx.draw_networkx_edges(G, pos, edgelist=G.edges, style='solid', width=1)
nx.draw_networkx_labels(G, pos)
#nx.draw_networkx_edge_labels(G,pos,edge_labels=nx.get_edge_attributes(G,'weight'))

print()

##Construct QUBOs

###Minimum cut

In [22]:
# ------- Deterministic minimum cut -------

cut_value, partition = stopper(lambda: nx.minimum_cut(G, 0, N, 'weight'))

print("Minimum cut value:", cut_value)
reachable, non_reachable = partition

#pos = nx.spring_layout(G)
#nx.draw_networkx_nodes(G, pos, nodelist=non_reachable, node_color='c')
#nx.draw_networkx_nodes(G, pos, nodelist=reachable, node_color='r')
#nx.draw_networkx_edges(G, pos)

Elapsed time: 4.9302308559417725
Minimum cut value: 241.52172259315452


In [13]:
# ------- Set up our QUBO dictionary MIN_CUT WEIGHTED-------

# Initialize our Q matrix
Q = defaultdict(float)

# Update Q matrix for every edge in the graph
for i, j in G.edges:
    w=G.edges[i,j]['weight']
    Q[(i,i)]+= 1*w
    Q[(j,j)]+= 1*w
    Q[(i,j)]+= -2*w

Q[(0,0)]+=1000000
Q[(N,N)]-=1000000

###Max-K-Cut

In [ ]:
# ------- Set up our QUBO dictionary MAX_K_CUT-------

# Initialize our Q matrix
Q = defaultdict(int)

# Update Q matrix for every edge in the graph
for i, j in G.edges:
  for k1 in range(K):
    for k2 in range(K):
      if (k1!=k2):
        #print("v_" + str(i) + "_" + str(k1) + " - " + "v_" + str(j) + "_" + str(k2))
        Q[(i*K+k1,j*K+k2)] += -1

for v in range(K*N):
  for k1 in range(K):
    for k2 in range(K):
      if (k1!=k2):
        Q[(v*K+k1,v*K+k2)] += 10000    

In [36]:
# ------- Set up our QUBO dictionary MAX_K_CUT WEIGHTED-------

# Initialize our Q matrix
Q = defaultdict(int)

# Update Q matrix for every edge in the graph
for u, v in G.edges:
  for i in range(K):
    for j in range(K):
      if (i!=j):
        w=G.edges[u,v]['weight']
        Q[(u*K+i,v*K+j)] += -1*w

for u in range(K*N):
  for i in range(K):
    for j in range(K):
      if (i!=j):
        Q[(u*K+i,u*K+j)] += 10000 

###Max-Cut

In [24]:
# ------- Set up our QUBO dictionary MAX_CUT-------

# Initialize our Q matrix
Q = defaultdict(int)

# Update Q matrix for every edge in the graph
for i, j in G.edges:
    Q[(i,i)]+= -1
    Q[(j,j)]+= -1
    Q[(i,j)]+= 2

In [25]:
# ------- Set up our QUBO dictionary MAX_CUT WEIGHTED-------

# Initialize our Q matrix
Q = defaultdict(float)

# Update Q matrix for every edge in the graph
for i, j in G.edges:
    w=G.edges[i,j]['weight']
    Q[(i,i)]+= -1*w
    Q[(j,j)]+= -1*w
    Q[(i,j)]+= 2*w

## Optimize QUBO

In [14]:
# ------- Run our QUBO on the CPU -------

# Run the QUBO on the solver from your config file
sampler = dimod.ExactSolver()
response = sampler.sample_qubo(Q)

In [37]:
# ------- Run our QUBO on the simulated QPU -------
# Set up QPU parameters
chainstrength = 8
numruns = 10

sampler = neal.SimulatedAnnealingSampler()
start = time.time()
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               label='Example - Maximum Cut weighted')
end = time.time()
print("Elapsed time:",end-start)

Elapsed time: 18.833839416503906


In [ ]:
# ------- Run our QUBO on hybrid -------

sampler = LeapHybridSampler()
start = time.time()
response = sampler.sample_qubo(Q,
                               label='Example - Maximum Cut weighted')
end = time.time()
print("Elapsed time:",end-start)

Elapsed time: 2.095048189163208


In [ ]:
# ------- Run our QUBO on the QPU -------
# Set up QPU parameters
chainstrength = 8
numruns = 10

sampler = EmbeddingComposite(DWaveSampler())
start = time.time()
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               label='Example - Maximum Cut weighted')
end = time.time()
print("Elapsed time:",end-start)

Elapsed time: 3.234708309173584


In [ ]:
# ------- Print results to user -------
print('-' * 60)
print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
print('-' * 60)
for sample, E in response.data(fields=['sample','energy']):
    S0 = [k for k,v in sample.items() if v == 0]
    S1 = [k for k,v in sample.items() if v == 1]
    print('{:>15s}{:>15s}{:^15s}{:^15s}'.format(str(S0),str(S1),str(E),str(int(-1*E))))


In [ ]:
# ------- Display results to user -------
# Grab best result
# Note: "best" result is the result with the lowest energy
# Note2: the look up table (lut) is a dictionary, where the key is the node index
#   and the value is the set label. For example, lut[5] = 1, indicates that
#   node 5 is in set 1 (S1).
lut = response.first.sample

# Interpret best result in terms of nodes and edges
S0 = [node for node in G.nodes if not lut[node]]
S1 = [node for node in G.nodes if lut[node]]
cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

# Display best result
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='r')
nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='c')
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=1)
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=1)
nx.draw_networkx_labels(G, pos)

filename = "maxcut_plot.png"
plt.savefig(filename, bbox_inches='tight')
print("\nYour plot is saved to {}".format(filename))

In [ ]:
# ------- Display results to user -------
# Grab best result
# Note: "best" result is the result with the lowest energy
# Note2: the look up table (lut) is a dictionary, where the key is the node index
#   and the value is the set label. For example, lut[5] = 1, indicates that
#   node 5 is in set 1 (S1).
lut = response.first.sample

# Interpret best result in terms of nodes and edges
color_map_found=[]

for v in range(K*N):
  for k in range(K):
    if (lut[v*K+k]==1):
      color_map_found.append(k)

cut_edges = [(u, v) for u, v in G.edges if color_map_found[u]!=color_map_found[v]]
uncut_edges = [(u, v) for u, v in G.edges if color_map_found[u]==color_map_found[v]]

# Display best result
#pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, nodelist=G.nodes, node_color=color_map_found)
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=1)
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=1)
#nx.draw_networkx_edges(G, pos, edgelist=G.edges, style='solid', width=1)
nx.draw_networkx_labels(G, pos)

filename = "maxcut_plot.png"
plt.savefig(filename, bbox_inches='tight')
print("\nYour plot is saved to {}".format(filename))

In [40]:
lut = response.first.sample

# Interpret best result in terms of nodes and edges
color_map_found=[]

for v in range(K*N):
  for k in range(K):
    if (lut[v*K+k]==1):
      color_map_found.append(k)

In [ ]:
color_map_found